In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
urls = ["https://www.icc-cricket.com/rankings/mens/player-rankings/odi",
        "https://www.icc-cricket.com/rankings/womens/player-rankings/odi",
        "https://www.icc-cricket.com/rankings/mens/team-rankings/odi",
        "https://www.icc-cricket.com/rankings/womens/team-rankings/odi"]


columns_team = ['Rank', 'Team', 'Matches', 'Points', 'Rating']
columns_player = ['Rank','Player','Team','Rating']

In [3]:
url = "https://www.icc-cricket.com/rankings/mens/player-rankings/odi"

In [4]:
def getSoup(url):
    content = requests.get(url).content
    # get soup
    soup = BeautifulSoup(content,'lxml') # lxml parser
    return soup

def create_df(data, columns):
    df = pd.DataFrame(data, columns=columns).set_index("Rank")
    return df

In [5]:
def get_team_rankings(url):
    soup = getSoup(url)
    table_bodies = soup.find('tbody')
    rows = table_bodies.find_all("tr")[:10]
    data = []
    for row in rows:
        temp = []
        for each in row.find_all('td'):
            temp.append(each.text.strip())                
        data.append(temp)
    for each in data:
        each[1] = each[1].split("\n")[0]
    return data

In [6]:
def extract_player_rankings(data_div):

    top_player = data_div.find_all("div", class_="rankings-block__banner")[0]
    other_players = data_div.find_all("table", class_="table")[0]
    
    top_player_data = top_player.find("div", class_="rankings-block__banner--player-info")
    
    temp = [1]
    for each in top_player_data.text.split("\n"):
        if len(each)<2:
            continue
        temp.append(each.strip())

    data = [temp]
    
    other_players_data = other_players.find_all('tbody')[0].find_all('tr')

    for each in other_players_data:
        data_ = each.find_all('td')
        temp = []
        for _ in data_:
            col_data = _.text.strip().split("\n")[0]
            temp.append(col_data)
        data.append(temp)
    return data

In [7]:
def get_player_rankings(url):
    soup = getSoup(url)

    data = soup.find_all("div", class_="u-negative-wrapper-tab uniform-grid touch-scroll-list--tablet")[0]
    
    if(url.split('/')[-3] == "mens"):
        data2 = data.find_all("div", class_="col-4 col-12-desk touch-scroll-list__element")
    else:
        data2 = data.find_all("div", class_="col-4 col-12-desk uniform-grid__section touch-scroll-list__element")
    
    batting_data = extract_player_rankings(data2[0])
    if(url.split('/')[-3] == "mens"):
        bowler_data = extract_player_rankings(data2[1])
        return batting_data, bowler_data
    else:
        all_rounder_data = extract_player_rankings(data2[2])
        return batting_data, all_rounder_data

In [8]:
def driver(urls):
    headings = []
    dfs = []
    for url in urls:
        if (url.split("/")[-2][:4] == "team"):
            heading = " ".join(url.split("/")[-3:])
            headings.append(heading)
            
            data = get_team_rankings(url)
            dfs.append(create_df(data, columns_team))
        else:
            heading1 = " ".join(url.split("/")[-3:]) + "-batting"
            heading2 = " ".join(url.split("/")[-3:]) + "-bowling/all_rounder"
            
            headings.append(heading1)
            headings.append(heading2)
    
            data1, data2 = get_player_rankings(url)  
            df1, df2 = create_df(data1, columns_player), create_df(data2, columns_player)
            dfs.append(df1)
            dfs.append(df2)
    return headings, dfs

In [9]:
headings, dfs = driver(urls)
for _ in range(len(headings)):
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")    
    print(headings[_])
    print("~~~~~~~~~~~~~~~~~")    
    print(dfs[_].head())
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
mens player-rankings odi-batting
~~~~~~~~~~~~~~~~~
            Player Team Rating
Rank                          
1       Babar Azam  PAK    873
2      Virat Kohli  IND    844
3     Rohit Sharma  IND    813
4      Ross Taylor   NZ    801
5      Aaron Finch  AUS    779
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
mens player-rankings odi-bowling/all_rounder
~~~~~~~~~~~~~~~~~
                Player Team Rating
Rank                              
1          Trent Boult   NZ    737
2       Josh Hazlewood  AUS    709
3     Mujeeb Ur Rahman  AFG    708
4         Chris Woakes  ENG    700
5         Mehedi Hasan  BAN    692
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
womens player-rankings odi-batting
~~~~~~~~~~~~~~~~~
               Player Team Rating
Rank                             
1         Mithali Raj  IND    762
2         Lizelle Lee   SA    758
3        Alyssa Healy  AUS    756
4      Tammy Beaumont